# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.random import RandomState
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

STATE = RandomState(12345)

<b> Загрузим данные геологоразведки трёх регионов </b>

In [2]:
data_0 = pd.read_csv('datasets/geo_data_0.csv')
data_1 = pd.read_csv('datasets/geo_data_1.csv')
data_2 = pd.read_csv('datasets/geo_data_2.csv')

<b> Данные геологоразведки 1-го региона </b>

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<b> Данные геологоразведки 2-го региона </b>

In [6]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


<b> Данные геологоразведки 3-го региона </b>

In [9]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [10]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


### Описание данных

Данные геологоразведки трёх регионов находятся в файлах:
* /datasets/geo_data_0.csv;
* /datasets/geo_data_1.csv;
* /datasets/geo_data_2.csv;
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

<b> Вывод: </b>
Данные успешно загружены и не требуют дополнительной предобработки.

## Обучение и проверка модели

In [12]:
def spit_and_train(data):
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    # разобьем данные на обучающую и валидационную выборки в соотношении 75:25
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size= 0.25, random_state= STATE)
    # применим масштабирование
    scaler = StandardScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    # модель линейной регрессии
    parameters = {'fit_intercept':[True,False], 'copy_X':[True, False], 'normalize':[True,False]}
    model = GridSearchCV(estimator=LinearRegression(), param_grid=parameters, cv=5)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    # посчитаем RMSE модели
    mse = mean_squared_error(target_valid, predictions_valid)
    rmse = mse ** 0.5
    # посчитаем средний запас предсказанного сырья
    average_stock = predictions_valid.mean()
    print('Cредний запас предсказанного сырья:', average_stock, '\nRMSE:', rmse, '\n')
    return predictions_valid, target_valid.reset_index(drop=True), average_stock

Сохраним предсказания и правильные ответы на валидационной выборке

In [13]:
print('1-й регион:')
predictions_0, target_valid_0, average_stock_0 = spit_and_train(data_0)
print('2-й регион:')
predictions_1, target_valid_1, average_stock_1 = spit_and_train(data_1)
print('3-й регион:')
predictions_2, target_valid_2, average_stock_2 = spit_and_train(data_2)

1-й регион:
Cредний запас предсказанного сырья: 92.59256778438035 
RMSE: 37.5794217150813 

2-й регион:
Cредний запас предсказанного сырья: 68.76995145799754 
RMSE: 0.8897367737680651 

3-й регион:
Cредний запас предсказанного сырья: 95.087528122523 
RMSE: 39.958042459521614 



<b> Вывод: </b>

Модели обучены и проверены на качество предсказания метрикой RMSE, которая определяет величину среднеквадратичной ошибки в предсказании модели. Во втором регионе величина RMSE сильно отличается от других.

## Подготовка к расчёту прибыли

* При разведке региона исследуем <b> 500 точек </b> , из которых с помощью машинного обучения выбирем <b> 200 </b>  лучших для разработки.
* Бюджет на разработку скважин в регионе — <b> 10 млрд рублей </b>.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет <b> 450 тыс. рублей </b>, поскольку объём указан в тысячах баррелей.
* После оценки рисков оставим лишь те регионы, в которых вероятность убытков меньше <b> 2.5% </b>. Среди них выбирем регион с наибольшей средней прибылью.

### Все ключевые значения для расчётов сохраним в отдельных переменных.

In [14]:
BUDGET = 10 ** 10
INCOME = 450 * 10 ** 3
POINTS = 500
BEST_POINTS = 200
PROBABILITY = 2.5 / 100

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [15]:
budget_per_one_point = BUDGET / POINTS
print('Бюджет на разработку одной скважины:', int(budget_per_one_point), 'рублей')

Бюджет на разработку одной скважины: 20000000 рублей


In [16]:
volume_stock = budget_per_one_point / INCOME
print('Достаточный объём сырья для безубыточной разработки новой скважины', round(volume_stock, 2), 'тыс. баррелей')

Достаточный объём сырья для безубыточной разработки новой скважины 44.44 тыс. баррелей


<b> Сравним полученный объём сырья со средним запасом в каждом регионе </b>

In [17]:
i = 0
for stock in [average_stock_0, average_stock_1, average_stock_2]:
    i+= 1
    print('Средняя доля ', i, '-го региона от достаточного объема сырья ниже на ', round(100 - stock / volume_stock * 100, 2), '%', sep='')

Средняя доля 1-го региона от достаточного объема сырья ниже на -108.33%
Средняя доля 2-го региона от достаточного объема сырья ниже на -54.73%
Средняя доля 3-го региона от достаточного объема сырья ниже на -113.95%


### Вывод по этапу подготовки расчёта прибыли

Достаточный объём сырья для безубыточной разработки новой скважины 111.11 тыс. баррелей. Средний запас сырья скважин в первом, втором и третьем регионе ниже необходимого для безубыточной разработки новой скважины на 16.77%, 37.89% и 14.52% соответственно.

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [18]:
def profit(predictions, target):
    target = target.reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    # отсорируем список по убыванию
    probs_sorted = predictions.sort_values(ascending=False)[:BEST_POINTS]
    selected = target[probs_sorted.index]
    # просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям
    sum_data_sorted = selected.sum()
    # рассчитаем прибыль для полученного объёма сырья
    return sum_data_sorted * INCOME - BUDGET

In [19]:
print('Прибыль по выбранным скважинам и предсказаниям модели 1-го региона:', round(profit(predictions_0, target_valid_0) / 1_000_000_000, 4), 'млрд. рублей')
print('Прибыль по выбранным скважинам и предсказаниям модели 2-го региона:', round(profit(predictions_1, target_valid_1) / 1_000_000_000, 4), 'млрд. рублей')
print('Прибыль по выбранным скважинам и предсказаниям модели 3-го региона:', round(profit(predictions_2, target_valid_2) / 1_000_000_000, 4), 'млрд. рублей')

Прибыль по выбранным скважинам и предсказаниям модели 1-го региона: 3.3208 млрд. рублей
Прибыль по выбранным скважинам и предсказаниям модели 2-го региона: 2.4151 млрд. рублей
Прибыль по выбранным скважинам и предсказаниям модели 3-го региона: 2.5399 млрд. рублей


## Расчёт рисков и прибыли для каждого региона

In [20]:
def get_values(predictions, target_valid):
    values = []
    for i in range(1000):
        target_subsample = target_valid.sample(n=POINTS, replace=True, random_state=STATE)
        probs_subsample = pd.Series(predictions)[target_subsample.index]
        values.append(profit(probs_subsample, target_subsample))
    values = pd.Series(values)
    lower = values.quantile(PROBABILITY)
    upper = values.quantile(1 - PROBABILITY)
    loss = sum(values < 0) / values.count() * 100
    mean = values.mean()
    print(f'Средняя прибыль: {round(mean / 1_000_000, 4)} млн.руб.')
    print(f'95%-й доверительный интервал: от {round(lower / 1_000_000, 4)} млн.руб. до {round(upper / 1_000_000, 4)} млн.руб.')
    print(f'Риск убытков: {loss}%\n')

In [21]:
print('1-й регион:')
get_values(predictions_0, target_valid_0)
print('2-й регион:')
get_values(predictions_1, target_valid_1)
print('3-й регион:')
get_values(predictions_2, target_valid_2)

1-й регион:
Средняя прибыль: 394.2844 млн.руб.
95%-й доверительный интервал: от -69.4506 млн.руб. до 915.4962 млн.руб.
Риск убытков: 6.1%

2-й регион:
Средняя прибыль: 454.7364 млн.руб.
95%-й доверительный интервал: от 61.0718 млн.руб. до 855.9141 млн.руб.
Риск убытков: 0.7000000000000001%

3-й регион:
Средняя прибыль: 353.6641 млн.руб.
95%-й доверительный интервал: от -162.6509 млн.руб. до 847.7618 млн.руб.
Риск убытков: 7.6%



<b> Вывод: </b>

В первом и третьем регионах риск убытков больше требуемых 2.5%. Таким образом, единственным подходящим будет второй регион, средняя прибыль в этом регионе составит 454.7364 млн.руб.